In [20]:
from datetime import datetime
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from gurobipy import *

## API Call + Data Preprocessing

In [21]:
# Create a list of symbols
symbols = ["NVDA", "GOOG", "AMZN", "AAPL", "META", "TSLA"]

daily_prices = yf.download(
  tickers = ' '.join(symbols), 
  start = datetime(2024, 1, 1),
  end = datetime(2025, 1, 1)
)['Adj Close']

daily_prices.columns = symbols

# Compute daily simple returns
daily_returns = (
  daily_prices.pct_change()
            .dropna(
              # Drop the first row since we have NaN's
              axis = 0,
              how = 'any',
              inplace = False
              )
)

# daily_returns = daily_returns.reset_index()
daily_returns

[*********************100%%**********************]  6 of 6 completed


,NVDA,GOOG,AMZN,AAPL,META,TSLA
Date,,,,,,
2024-01-03,-0.007488,-0.009738,0.005732,-0.005256,-0.012436,-0.040134
2024-01-04,-0.012700,-0.026268,-0.016529,0.007693,0.009018,-0.002181
2024-01-05,-0.004013,0.004634,-0.004709,0.013914,0.022897,-0.001849
2024-01-08,0.024175,0.026577,0.022855,0.019065,0.064281,0.012464
2024-01-09,-0.002263,0.015225,0.014445,-0.003429,0.016975,-0.022832
...,...,...,...,...,...,...
2024-12-24,0.011478,0.017729,0.008062,0.013170,0.003938,0.073572
2024-12-26,0.003176,-0.008732,-0.002379,-0.007240,-0.002068,-0.017630
2024-12-27,-0.013242,-0.014534,-0.015525,-0.005867,-0.020868,-0.049479


## Optimizing Portfolio
Optimizing the stock portfolio strategy by using the historical volatility and average daily return to minimize risk while ensuring a daily return of at least 0.25%.

In [32]:
# Mean returns and covariance matrix
mean_returns = daily_returns.mean()
cov_matrix = np.cov(daily_returns.T) # np.cov expects rows to represent variables so .T

# Initialize the model
m = Model("Portfolio")

# Create variables (bounded from 0 to 1 for the proportion each stock represents in portfolio)
weights = m.addVars(len(symbols), lb=0, ub=1, name="weights")

# Set the objective: maximize Sharpe ratio (minimize its negative)
m.setObjective(quicksum(
    weights[i] * cov_matrix[i, j] * weights[j] for i in range(len(symbols)) for j in range(len(symbols))
    ), GRB.MINIMIZE)

# Add constraint: weights must sum to 1
m.addConstr(quicksum(weights[i] for i in range(len(symbols))) == 1, name="WeightSum")

# Optimize the model
m.optimize()

# Print results
if m.status == GRB.OPTIMAL:
    print("Optimal Sharpe Ratio Achieved")
    print("Portfolio Weights:")
    for i, symbol in enumerate(symbols):
        print(f"{symbol}: {weights[i].x}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 6 columns and 6 nonzeros
Model fingerprint: 0x75bf0524
Model has 21 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [3e-04, 3e-03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 1 rows, 6 columns, 6 nonzeros
Presolved model has 21 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 5
 AA' NZ     : 1.500e+01
 Factor NZ  : 2.100e+01
 Factor Ops : 9.100e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.50119186e

In [28]:
mean_returns

NVDA    0.001312
GOOG    0.001674
AMZN    0.001405
AAPL    0.002365
META    0.004634
TSLA    0.002721
dtype: float64

In [ ]:
daily_returns.mean()

Date
2024-01-03   -0.011553
2024-01-04   -0.006828
2024-01-05    0.005146
2024-01-08    0.028236
2024-01-09    0.003020
                ...   
2024-12-24    0.021325
2024-12-26   -0.005812
2024-12-27   -0.019919
2024-12-30   -0.012494
2024-12-31   -0.015474
Length: 251, dtype: float64